In [ ]:
import requests
import pandas as pd

# Konfigurasi API ThingSpeak
channel_id = 2326256
read_api_key = 'BKIJNOYDM205J2HG'
url = f'https://api.thingspeak.com/channels/{channel_id}/feeds.json'

# Parameter untuk permintaan API
params = {
    'api_key': read_api_key,
    'results': 1440  # Jumlah data yang ingin diambil, bisa disesuaikan
}

# Mengirim permintaan GET ke ThingSpeak
response = requests.get(url, params=params)

# Cek apakah permintaan berhasil
if response.status_code == 200:
    # Mengambil data feed dalam format JSON
    data = response.json()
    feeds = data['feeds']
    
    # Membuat DataFrame Pandas dari data feed
    df = pd.DataFrame(feeds)
    
    # Pilih kolom-kolom yang ingin disimpan, misalnya waktu dan field1 hingga field8
    df = df[['created_at', 'field1', 'field2', 'field3', 'field4', 'field8']]
    
    # Ganti nama kolom untuk lebih deskriptif
    df.columns = ['Time', 'Suhu', 'Kelembapan', 'Tekanan', 'Embun', 'Volt']
    
    # Simpan DataFrame ke file CSV
    df.to_csv('Cuaca.csv', index=False)
    
    # Tampilkan DataFrame untuk pengecekan
    print(df.head())
else:
    print(f"Failed to fetch data. HTTP Status code: {response.status_code}")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>